# SIIM-FISABIO-RSNA COVID-19 Detection Opacity Localization
Team ML Ballers

## Resources:
- Competition Page: https://www.kaggle.com/c/siim-covid19-detection
- Dataset Info: https://arxiv.org/pdf/2006.01174.pdf
- Annotation Info: https://journals.lww.com/thoracicimaging/Fulltext/2020/11000/Review_of_Chest_Radiograph_Findings_of_COVID_19.4.aspx
- Emsembling methods: https://towardsdatascience.com/ensemble-methods-in-machine-learning-what-are-they-and-why-use-them-68ec3f9fef5f
- What everyone else is doing: https://www.kaggle.com/pvtien96/siim-cov19-efnb7-yolov5-infer
- Interesting ideas: https://www.kaggle.com/davidbroberts/lung-segmentation-without-cnn

In [ ]:
import os
import numpy as np
import pandas as pd
import json
import pydicom as dicom
import matplotlib.pylab as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
from PIL import Image
import time

# from tensorflow.keras.applications.EfficientNetB7 import EfficientNetB7

## Getting Data

In [ ]:
image_level = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
first_image = image_level.iloc[0]
first_image

image_level

In [ ]:
study_level = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
study_level

## Data Coolification

In [ ]:
study_level.loc[study_level['Typical Appearance'] == 1, 'Class'] = 'typical'
study_level.loc[study_level['Negative for Pneumonia'] == 1, 'Class'] = 'negative'
study_level.loc[study_level['Indeterminate Appearance'] == 1, 'Class'] = 'indeterminate'
study_level.loc[study_level['Atypical Appearance'] == 1, 'Class'] = 'atypical'

study_level['StudyInstanceUID'] = study_level['id'].str[:-6]

study_level

In [ ]:
image_level = image_level.merge(study_level[['StudyInstanceUID', 'Class']], on='StudyInstanceUID')

image_level

## Localization Time

Options:
- Follow this tutorial for yolov5: https://github.com/ultralytics/yolov5
- Transfer learning with one of these models: https://keras.io/api/applications/
  - Example: https://www.kaggle.com/niloofarrahmani/transfer-learning-with-mobilenet-v2

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!mv yolov5/* ./

In [ ]:
!ls

In [ ]:
!ls data/

In [ ]:
!cat data/coco.yaml

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l.pt

In [ ]:
!ls models/

In [ ]:
!cat models/yolov5l.yaml

In [ ]:
!pip install -r requirements.txt

In [ ]:
!mkdir opacities
!mkdir opacities/images
!mkdir opacities/images/train
!mkdir opacities/images/val
!mkdir opacities/images/test
!mkdir opacities/labels
!mkdir opacities/labels/train
!mkdir opacities/labels/val
!mkdir opacities/labels/test

In [ ]:
!ls opacities/images/train
!pwd

In [ ]:
!rm -rf opacities/images/train/*

In [ ]:
IMAGE_SIZE = 640

def save_jpg_from_dicom(dicom_img, path, img_id):
    data = dicom.pixel_data_handlers.util.apply_voi_lut(dicom_img.pixel_array, dicom_img)
    
    if dicom_img.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data -= np.min(data)
    data = data /np.max(data)
    data = (data * 255).astype(np.uint8)
    
    image = Image.fromarray(data)
    
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    
    outfile = f"{path}im{img_id}.jpg"
    
    try:
        image.save(outfile)
    except OSError:
        print("cannot convert")

In [ ]:
def save_bounding_box_from_image(image_shape, boxes, path, img_id):
    save_string = ''
    
    if str(boxes) == 'nan':
        boxes = []
    else: 
        boxes = str(boxes).replace("'", '"')
        boxes = json.loads(boxes)
    
    for box in boxes:
        if len(save_string) > 0:
            save_string = f"{save_string}\n"
        
        width = int(box['width']) / image_shape[1]
        height = int(box['height']) / image_shape[0]
        center_x = (int(box['x']) / image_shape[1]) + width
        center_y = (int(box['y']) / image_shape[0]) + height
        
        save_string = f"{save_string}0 {center_x} {center_y} {width} {height}"
    
    outfile = f"{path}im{img_id}.txt"
    
    with open(outfile, "w") as text_file:
        text_file.write(save_string)

In [ ]:
training_image_directory = '../input/siim-covid19-detection/train/'
opacities_directory = 'opacities/'

count = 0

for subdir, dirs, files in os.walk(training_image_directory):
    for filename in files:
        filepath = subdir + os.sep + filename
        
        count += 1
        
        print(f'{count} out of {6334}', end='\r')

        if filepath.endswith(".dcm"):
            row = image_level.loc[image_level['id'] == f'{filepath.split("/")[-1][:-4]}_image']
            
            try:
                if count < 5500:
                    data_step = 'train'
                elif count < 6000:
                    data_step = 'val'
                else:
                    data_step = 'test'
                
                save_jpg_from_dicom(dicom.dcmread(filepath), f'{opacities_directory}images/{data_step}/', row['id'].values[0])
                save_bounding_box_from_image(dicom.dcmread(filepath).pixel_array.shape, row['boxes'].values[0], f'{opacities_directory}labels/{data_step}/', row['id'].values[0])
            except Exception as e:
                pass
        

In [ ]:
!ls opacities/images/train
!ls opacities/labels/train

In [ ]:
import matplotlib.image as mpimg

IMAGE_NUMBER = 21

# ../input/siim-covid19-detection/train/ae3e63d94c13/288554eb6182/e00f9fe0cce5.dcm

img = mpimg.imread(f'opacities/images/train/im{IMAGE_NUMBER}.jpg')
# imgplot = plt.imshow(img)

fig, ax = plt.subplots()
ax.imshow(img)

with open(f'opacities/labels/train/im{IMAGE_NUMBER}.txt', "r") as a_file:
    for line in a_file:
        stripped_line = line.strip()
        box = stripped_line.split()
        
        rect = patches.Rectangle(((float(box[1]) - float(box[3])) * IMAGE_SIZE, 
                                  (float(box[2]) - float(box[4])) * IMAGE_SIZE), 
                                 float(box[3]) * IMAGE_SIZE, 
                                 float(box[4]) * IMAGE_SIZE, 
                                linewidth=2, edgecolor='r', facecolor='none')
        
        ax.add_patch(rect)

plt.show()

In [ ]:
!cat opacities/labels/train/im1.txt

## YOLO

In [ ]:
# opacities.yaml
outfile = 'opacities.yaml'

save_string = ["path: opacities/",
              "train: images/train",
              "val: images/val",
              "test: images/test",
              "nc: 1",
              "names: ['opacity']"]

save_string = "\n".join(save_string)

with open(outfile, "w") as text_file:
    text_file.write(save_string)

In [ ]:
yaml_file = 'models/yolov5l.yaml'
outfile = 'custom_yolov5l.yaml'

save_string = ''

with open(yaml_file, "r") as text_file:
    save_string = text_file.read()
    
save_string = save_string.replace("80", "1", 1)
print(save_string)
  
    
with open(outfile, "w") as text_file:
    text_file.write(save_string)

In [ ]:
!wandb off
!python3 train.py --img 640 --batch 16 --epochs 40 --data opacities.yaml --cfg custom_yolov5l.yaml --weights yolov5l.pt

## Save weights in Firebase

In [ ]:
!pip install firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

if not firebase_admin._apps:
    cred = credentials.Certificate('[INSERT PATH TO FIREBASE SECRETS FILE]')
    firebase_admin.initialize_app(cred, {
        'storageBucket': '[INSERT STORAGE BUCKET URL]'
    })

file_path = 'runs/train/exp/weights/'
file_name = 'best.pt'

# file_path = ''
# file_name = 'requirements.txt'

bucket = storage.bucket()
blob = bucket.blob(f'{file_name[:-3]}{int(time.time())}{file_name[-3:]}')

blob.upload_from_filename(f'{file_path}{file_name}')

## Testing Detection

In [ ]:
import matplotlib.image as mpimg

for i in range(10):
    img = mpimg.imread(f'runs/detect/exp4/im{i}.jpg')
    # print(dicom.dcmread(filepath).pixel_array)
    imgplot = plt.imshow(img)

    plt.show()

In [ ]:
!head runs/train/exp3/weights/best.pt

In [ ]:
!python detect.py --weights runs/train/exp5/weights/best.pt --img 640 --source opacities/images/train --save-txt --save-conf --conf-thres 0.05

In [ ]:
!ls -lah runs/detect/exp15/labels/
!cat runs/detect/exp15/labels/im7c40e04c6163_image.txt